In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('reduced_data_to_plot_7.csv', index_col=False)
top20 = pd.read_csv('top20.csv', index_col=0)

In [4]:
df.head()

,start_station_name,end_station_name,date,avgTemp,value,bike_rides_daily,season
0,West End Ave & W 78 St,W 44 St & 11 Ave,2022-01-01,11.6,1,20895,winter
1,Willoughby Ave & Hall St,Willoughby St & Fleet St,2022-01-01,11.6,1,20895,winter
2,Crescent St & 34 Ave,Crescent St & 30 Ave,2022-01-01,11.6,1,20895,winter
3,Central Park West & W 85 St,5 Ave & E 78 St,2022-01-01,11.6,1,20895,winter
4,W 30 St & 10 Ave,11 Ave & W 27 St,2022-01-01,11.6,1,20895,winter


In [11]:
top20.head()

,start_station_name,value
1587,W 21 St & 6 Ave,131679
1718,West St & Chambers St,126290
495,Broadway & W 58 St,117797
286,6 Ave & W 33 St,109542
8,1 Ave & E 68 St,108412


In [9]:
df_groupby_bar = df.groupby('start_station_name', as_index=False).agg({'value': 'sum'})
top20 = df_groupby_bar.nlargest(20, 'value')

In [16]:
df_route_popularity = df.groupby(['start_station_name','end_station_name'], as_index=False).agg({'value': 'sum'})
top50_routes = df_route_popularity.nlargest(50, 'value')

In [15]:
top50_routes.head(50)

,end_station_name,start_station_name,value
96637,Central Park S & 6 Ave,Central Park S & 6 Ave,754
45267,7 Ave & Central Park South,7 Ave & Central Park South,530
188330,Grand Army Plaza & Central Park S,Grand Army Plaza & Central Park S,466
267079,Roosevelt Island Tramway,Roosevelt Island Tramway,441
83823,Broadway & W 58 St,Broadway & W 58 St,423
344666,West St & Chambers St,West St & Chambers St,419
81866,Broadway & W 41 St,Broadway & W 41 St,393
36628,5 Ave & E 72 St,5 Ave & E 72 St,387
273412,Soissons Landing,Soissons Landing,366
9037,12 Ave & W 40 St,12 Ave & W 40 St,361


In [17]:
df_route_popularity['Trip_Type'] = np.where(df_route_popularity['start_station_name'] == df_route_popularity['end_station_name'], 'Round-Trip', 'Point-to-Point')

In [48]:
# Get the top 50 routes by value
top50_routes = df_route_popularity.nlargest(50, 'value')

# Create a new column 'route' that concatenates start and end station names
top50_routes['Route'] = top50_routes['start_station_name'] + ' > ' + top50_routes['end_station_name']

# Drop the 'start_station_name' and 'end_station_name' columns
top50_routes = top50_routes.drop(columns=['start_station_name', 'end_station_name'])

new_order = ['Route', 'Trip_Type', 'value']
top50_routes = top50_routes[new_order]

#Generate simple csvs to make streamlit dash generation more simplistic.
top_round_routes = top50_routes[top50_routes['Trip_Type']=='Round-Trip']
top_point_routes = top50_routes[top50_routes['Trip_Type']=='Point-to-Point']

top_round_routes.to_csv('top_round_routes.csv')
top_point_routes.to_csv('top_point_routes.csv')

In [49]:
top_point_routes.head(10)

,Route,Trip_Type,value
305377,W 21 St & 6 Ave > 9 Ave & W 22 St,Point-to-Point,297
248476,Old Slip & South St > South St & Gouverneur Ln,Point-to-Point,259
2890,1 Ave & E 62 St > 1 Ave & E 68 St,Point-to-Point,250
247256,North Moore St & Greenwich St > Vesey St & Chu...,Point-to-Point,236
305753,W 21 St & 6 Ave > W 22 St & 10 Ave,Point-to-Point,234
351717,Yankee Ferry Terminal > Soissons Landing,Point-to-Point,222
272578,Soissons Landing > Yankee Ferry Terminal,Point-to-Point,219
200631,Henry St & Grand St > Norfolk St & Broome St,Point-to-Point,218
246555,Norfolk St & Broome St > Henry St & Grand St,Point-to-Point,214
96171,Central Park S & 6 Ave > 5 Ave & E 87 St,Point-to-Point,213


In [50]:
top_round_routes.head(10)

,Route,Trip_Type,value
96233,Central Park S & 6 Ave > Central Park S & 6 Ave,Round-Trip,754
45078,7 Ave & Central Park South > 7 Ave & Central P...,Round-Trip,530
188234,Grand Army Plaza & Central Park S > Grand Army...,Round-Trip,466
266323,Roosevelt Island Tramway > Roosevelt Island Tr...,Round-Trip,441
83687,Broadway & W 58 St > Broadway & W 58 St,Round-Trip,423
344632,West St & Chambers St > West St & Chambers St,Round-Trip,419
81691,Broadway & W 41 St > Broadway & W 41 St,Round-Trip,393
36288,5 Ave & E 72 St > 5 Ave & E 72 St,Round-Trip,387
272574,Soissons Landing > Soissons Landing,Round-Trip,366
8748,12 Ave & W 40 St > 12 Ave & W 40 St,Round-Trip,361


In [51]:
# Group by 'Trip_Type' and sum the 'value' column
result = top50_routes.groupby('Trip_Type')['value'].sum().reset_index()

# Display the result
print(result)

        Trip_Type  value
0  Point-to-Point   3610
1      Round-Trip  10383


In [52]:
result.to_csv('aggregated_trips.csv')

PermissionError: [Errno 13] Permission denied: 'aggregated_trips.csv'

In [53]:
value_A = result.loc[result['Trip_Type'] == 'Point-to-Point', 'value'].values[0]
value_B = result.loc[result['Trip_Type'] == 'Round-Trip', 'value'].values[0]

In [44]:
value_A

np.int64(3610)